# Debugging Notbook
If the debugger is attached, we can use the debugger to touch vscode debug points, when calling routes.

In [1]:
import httpx
import environ
import importlib
from pathlib import Path
from rich import print

In [2]:
# ! Set the environment mode you want to use here
# MODE = "production"
MODE = "develop"

In [3]:
# parent directory cwd for incomplicated module level imports
cwd = Path.cwd()
if not list(cwd.glob("manage.py")):
    cwd = cwd.parent
%cd $cwd

# load environment variables depending on mode
env = environ.Env()
envspath = cwd / "envs"
env.read_env(envspath / f"{MODE}.env")
DJANGO_SETTINGS_MODULE = env.str("DJANGO_SETTINGS_MODULE")
HOST = env('PROJECT_FQDN')

# load django settings, this is a bit complicated doing it manually because we
# inject a single settings (the TRUTH object) - built from all settings consecutively
# into the django settings module
from django.conf import global_settings as local_settings
base_settings = importlib.import_module('config.settings.base')
mode_settings = importlib.import_module(DJANGO_SETTINGS_MODULE)
for attribute in dir(base_settings):
    setattr(local_settings, attribute, getattr(base_settings, attribute))
for attribute in dir(mode_settings):
    setattr(local_settings, attribute, getattr(mode_settings, attribute))

from django.conf import settings
try:
    settings.configure(default_settings=local_settings)
except RuntimeError:
    pass

# now this works as excpeted and django apps are loaded..
import django
django.setup()

# load urls routes..
from io import StringIO
output = StringIO()
from django.core.management import call_command
_ = call_command('show_urls', stdout=output)  # from django_extensions.management.commands import show_urls

urls = []
for string in output.getvalue().split('\n'):
    route, *_ = string.split('\t')
    if not route.startswith('/_') and '<path>' not in route:
        urls.append(f"{HOST}{route}")
urls = list(set(urls))

c:\Users\D4662805\Documents\workspace\django-authentication-template


In [4]:
# create a user
email = "user@example.com"
password = "secret"
delete_all_existing = True

from django.contrib.auth import get_user_model
from django.db import IntegrityError
UserModel = get_user_model()
from asgiref.sync import sync_to_async

def create_user(email, password, delete_all_existing: bool = False):
    if delete_all_existing:
        UserModel.objects.all().delete()
    try:
        user = UserModel.objects.create(username=email, email=email)
    except IntegrityError:
        user = UserModel.objects.get(email=email)
    user.set_password(password)
    user.save()
    return user

user = await sync_to_async(create_user)(email, password, delete_all_existing=delete_all_existing)

In [5]:
# all the urls available for django
print(urls)

[
    'http://localhost:8000/api/protected/',
    'http://localhost:8000',
    'http://localhost:8000/api/auth/token/',
    'http://localhost:8000/api/'
]

In [6]:
url = [url for url in urls if "auth/token" in url][0]  # login

r = httpx.post(url, json={"email": email, "password": password})
print(f"{url} - {r.status_code}")


ReadTimeout: timed out

In [ ]:
print(r.json())
print(r.cookies)